In [0]:
from pyspark.sql.functions import split
from pyspark.sql.functions import lit, col, column, expr, udf, when
from pyspark.sql.functions import datediff, to_date
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime
from functools import reduce
import pyspark.sql.functions as F
from pyspark.sql.functions import current_date
from pyspark.sql.types import IntegerType,BooleanType,DateType
from pyspark.sql.functions import from_unixtime, unix_timestamp
from pyspark.sql.functions import split


In [0]:
path_dataset = "/FileStore/tables/OriginaisNetflix.parquet"

In [0]:
file_type = "parquet"
infer_schema = "true"
first_row_is_header = "true"
delimiter = ";"

df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(path_dataset)\
# df.show()
display(df)

Title,Genre,GenreLabels,Premiere,Seasons,SeasonsParsed,EpisodesParsed,Length,MinLength,MaxLength,Status,Active,Table,Language,dt_inclusao
House of Cards,Political drama,"political,drama",1-Feb-13,"6 seasons, 73 episodes",6,73,42–59 min.,42,59,Ended,0,Drama,English,2021-03-16T21:20:24.167-03:00
Hemlock Grove,Horror/thriller,"horror,thriller",19-Apr-13,"3 seasons, 33 episodes",3,33,45–58 min.,45,58,Ended,0,Drama,English,2021-03-16T21:20:24.167-03:00
Orange Is the New Black,Comedy-drama,comedy-drama,11-Jul-13,"6 seasons, 78 episodes",6,78,50–92 min.,50,92,Renewed,1,Drama,English,2021-03-16T21:20:24.167-03:00
Marco Polo,Historical drama,"historical,drama",12-Dec-14,"2 seasons, 20 episodes",2,20,48–65 min.,48,65,Ended,0,Drama,English,2021-03-16T21:20:24.167-03:00
Bloodline,Thriller,thriller,20-Mar-15,"3 seasons, 33 episodes",3,33,48–68 min.,48,68,Ended,0,Drama,English,2021-03-16T21:20:24.167-03:00
Sense8,Science fiction,science-fiction,5-Jun-15,"2 seasons, 24 episodes",2,24,45–152 min.,45,152,Ended,0,Drama,English,2021-03-16T21:20:24.167-03:00
Narcos,Crime drama,"crime,drama",28-Aug-15,"3 seasons, 30 episodes",3,30,43–60 min.,43,60,Ended,0,Drama,English,2021-03-16T21:20:24.167-03:00
Stranger Things,Science fiction/horror,"science-fiction,horror",15-Jul-16,"2 seasons, 17 episodes",2,17,42–62 min.,42,62,Renewed,1,Drama,English,2021-03-16T21:20:24.167-03:00
The Get Down,Musical drama,"musical,drama",12-Aug-16,"2 parts, 11 episodes",0,11,50–93 min.,50,93,Ended,0,Drama,English,2021-03-16T21:20:24.167-03:00
The Crown,Historical drama,"historical,drama",4-Nov-16,"2 seasons, 20 episodes",2,20,54–61 min.,54,61,Renewed,1,Drama,English,2021-03-16T21:20:24.167-03:00


In [0]:
df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- GenreLabels: string (nullable = true)
 |-- Premiere: string (nullable = true)
 |-- Seasons: string (nullable = true)
 |-- SeasonsParsed: long (nullable = true)
 |-- EpisodesParsed: long (nullable = true)
 |-- Length: string (nullable = true)
 |-- MinLength: long (nullable = true)
 |-- MaxLength: long (nullable = true)
 |-- Status: string (nullable = true)
 |-- Active: long (nullable = true)
 |-- Table: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- dt_inclusao: string (nullable = true)



In [0]:
display(df)

Title,Genre,GenreLabels,Premiere,Seasons,SeasonsParsed,EpisodesParsed,Length,MinLength,MaxLength,Status,Active,Table,Language,dt_inclusao
House of Cards,Political drama,"political,drama",1-Feb-13,"6 seasons, 73 episodes",6,73,42–59 min.,42,59,Ended,0,Drama,English,2021-03-16T21:20:24.167-03:00
Hemlock Grove,Horror/thriller,"horror,thriller",19-Apr-13,"3 seasons, 33 episodes",3,33,45–58 min.,45,58,Ended,0,Drama,English,2021-03-16T21:20:24.167-03:00
Orange Is the New Black,Comedy-drama,comedy-drama,11-Jul-13,"6 seasons, 78 episodes",6,78,50–92 min.,50,92,Renewed,1,Drama,English,2021-03-16T21:20:24.167-03:00
Marco Polo,Historical drama,"historical,drama",12-Dec-14,"2 seasons, 20 episodes",2,20,48–65 min.,48,65,Ended,0,Drama,English,2021-03-16T21:20:24.167-03:00
Bloodline,Thriller,thriller,20-Mar-15,"3 seasons, 33 episodes",3,33,48–68 min.,48,68,Ended,0,Drama,English,2021-03-16T21:20:24.167-03:00
Sense8,Science fiction,science-fiction,5-Jun-15,"2 seasons, 24 episodes",2,24,45–152 min.,45,152,Ended,0,Drama,English,2021-03-16T21:20:24.167-03:00
Narcos,Crime drama,"crime,drama",28-Aug-15,"3 seasons, 30 episodes",3,30,43–60 min.,43,60,Ended,0,Drama,English,2021-03-16T21:20:24.167-03:00
Stranger Things,Science fiction/horror,"science-fiction,horror",15-Jul-16,"2 seasons, 17 episodes",2,17,42–62 min.,42,62,Renewed,1,Drama,English,2021-03-16T21:20:24.167-03:00
The Get Down,Musical drama,"musical,drama",12-Aug-16,"2 parts, 11 episodes",0,11,50–93 min.,50,93,Ended,0,Drama,English,2021-03-16T21:20:24.167-03:00
The Crown,Historical drama,"historical,drama",4-Nov-16,"2 seasons, 20 episodes",2,20,54–61 min.,54,61,Renewed,1,Drama,English,2021-03-16T21:20:24.167-03:00


In [0]:
# 1. Transformar os campos "Premiere" e "dt_inclusao" de string para datetime.

df1 = df.withColumn("Premiere1_Dia", split(col("Premiere"), "-").getItem(0))\
.withColumn("Premiere2_Mes", split(col("Premiere"), "-").getItem(1))\
.withColumn("Premiere3_Ano", split(col("Premiere"), "-").getItem(2))

df_1 = df1.withColumn('Premiere2_Mes', 
                      when(df1.Premiere2_Mes.endswith('Jan'),regexp_replace(df1.Premiere2_Mes,'Jan','01')) \
                     .when(df1.Premiere2_Mes.endswith('Feb'),regexp_replace(df1.Premiere2_Mes,'Feb','02')) \
                     .when(df1.Premiere2_Mes.endswith('Mar'),regexp_replace(df1.Premiere2_Mes,'Mar','03')) \
                     .when(df1.Premiere2_Mes.endswith('Apr'),regexp_replace(df1.Premiere2_Mes,'Apr','04')) \
                     .when(df1.Premiere2_Mes.endswith('May'),regexp_replace(df1.Premiere2_Mes,'May','05')) \
                     .when(df1.Premiere2_Mes.endswith('Jun'),regexp_replace(df1.Premiere2_Mes,'Jun','06')) \
                     .when(df1.Premiere2_Mes.endswith('Jul'),regexp_replace(df1.Premiere2_Mes,'Jul','07')) \
                     .when(df1.Premiere2_Mes.endswith('Aug'),regexp_replace(df1.Premiere2_Mes,'Aug','08')) \
                     .when(df1.Premiere2_Mes.endswith('Sep'),regexp_replace(df1.Premiere2_Mes,'Sep','09')) \
                     .when(df1.Premiere2_Mes.endswith('Oct'),regexp_replace(df1.Premiere2_Mes,'Oct','10')) \
                     .when(df1.Premiere2_Mes.endswith('Nov'),regexp_replace(df1.Premiere2_Mes,'Nov','11')) \
                     .when(df1.Premiere2_Mes.endswith('Dec'),regexp_replace(df1.Premiere2_Mes,'Dec','12')).otherwise(df1.Premiere2_Mes))

df_11 = df_1.withColumn("FullPremiere",concat(col("Premiere1_Dia"),lit('-'),col("Premiere2_Mes"),lit('-'),col("Premiere3_Ano")))

#df_111 = df_11.withColumn("FullPremiere", F.to_timestamp('Date_Time', 'dd/MM/yyyy hh:mm:ss a'))

#df_111 = df_11.withColumn('FullPremiere', to_timestamp(current_timestamp(), "MM-dd-yyyy hh mm a"))


df_111 = df_11.withColumn("Premiere",col("Premiere").cast(DateType())) \
    .withColumn("dt_inclusao",col("dt_inclusao").cast(DateType())) \
    .withColumn("FullPremiere",col("FullPremiere").cast(DateType()))

#df_111 = df_11.withColumn("FullPremiere",to_timestamp(col("FullPremiere")))


display(df_111)


Title,Genre,GenreLabels,Premiere,Seasons,SeasonsParsed,EpisodesParsed,Length,MinLength,MaxLength,Status,Active,Table,Language,dt_inclusao,Premiere1_Dia,Premiere2_Mes,Premiere3_Ano,FullPremiere
House of Cards,Political drama,"political,drama",null,"6 seasons, 73 episodes",6,73,42–59 min.,42,59,Ended,0,Drama,English,2021-03-16,1,02,13,null
Hemlock Grove,Horror/thriller,"horror,thriller",null,"3 seasons, 33 episodes",3,33,45–58 min.,45,58,Ended,0,Drama,English,2021-03-16,19,04,13,null
Orange Is the New Black,Comedy-drama,comedy-drama,null,"6 seasons, 78 episodes",6,78,50–92 min.,50,92,Renewed,1,Drama,English,2021-03-16,11,07,13,null
Marco Polo,Historical drama,"historical,drama",null,"2 seasons, 20 episodes",2,20,48–65 min.,48,65,Ended,0,Drama,English,2021-03-16,12,12,14,null
Bloodline,Thriller,thriller,null,"3 seasons, 33 episodes",3,33,48–68 min.,48,68,Ended,0,Drama,English,2021-03-16,20,03,15,null
Sense8,Science fiction,science-fiction,null,"2 seasons, 24 episodes",2,24,45–152 min.,45,152,Ended,0,Drama,English,2021-03-16,5,06,15,null
Narcos,Crime drama,"crime,drama",null,"3 seasons, 30 episodes",3,30,43–60 min.,43,60,Ended,0,Drama,English,2021-03-16,28,08,15,null
Stranger Things,Science fiction/horror,"science-fiction,horror",null,"2 seasons, 17 episodes",2,17,42–62 min.,42,62,Renewed,1,Drama,English,2021-03-16,15,07,16,null
The Get Down,Musical drama,"musical,drama",null,"2 parts, 11 episodes",0,11,50–93 min.,50,93,Ended,0,Drama,English,2021-03-16,12,08,16,null
The Crown,Historical drama,"historical,drama",null,"2 seasons, 20 episodes",2,20,54–61 min.,54,61,Renewed,1,Drama,English,2021-03-16,4,11,16,null


In [0]:
df_111.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- GenreLabels: string (nullable = true)
 |-- Premiere: date (nullable = true)
 |-- Seasons: string (nullable = true)
 |-- SeasonsParsed: long (nullable = true)
 |-- EpisodesParsed: long (nullable = true)
 |-- Length: string (nullable = true)
 |-- MinLength: long (nullable = true)
 |-- MaxLength: long (nullable = true)
 |-- Status: string (nullable = true)
 |-- Active: long (nullable = true)
 |-- Table: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- dt_inclusao: date (nullable = true)
 |-- Premiere1_Dia: string (nullable = true)
 |-- Premiere2_Mes: string (nullable = true)
 |-- Premiere3_Ano: string (nullable = true)
 |-- FullPremiere: date (nullable = true)



In [0]:
# 2. Ordenar os dados por ativos e gênero de forma decrescente, 0 = inativo e 1 = ativo, todos com número 1 devem aparecer primeiro.

df2 = df_11.sort(col("Active").desc(),col("Genre").desc())
display(df2)

Title,Genre,GenreLabels,Premiere,Seasons,SeasonsParsed,EpisodesParsed,Length,MinLength,MaxLength,Status,Active,Table,Language,dt_inclusao,Premiere1,Premiere2,Premiere3,FullPremiere
True Tunes,chrildrens musical short,"chrildrens,musical,short",12-Jul-19,TBA,0,0,TBA,0,0,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,12,07,19,12-07-19
Super Monsters Monster Party,chrildrens musical short,"chrildrens,musical,short",14-Sep-18,"1 season, 4 episodes",1,4,2 min.,2,2,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,14,09,18,14-09-18
Dance & Sing with True,chrildrens musical short,"chrildrens,musical,short",18-May-18,"1 season, 11 episodes",1,11,2 min.,2,2,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,18,05,18,18-05-18
Trolls: The Beat Goes On!,childrens-animation,childrens-animation,19-Jan-18,"6 seasons, 38 episodes",6,38,24–25 min.,24,25,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,19,01,18,19-01-18
Kulipari: Dream Walker,childrens-animation,childrens-animation,20-Nov-18,"1 season, 10 episodes",1,10,23 min.,23,23,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,20,11,18,20-11-18
Llama Llama,childrens-animation,childrens-animation,26-Jan-18,"1 season, 15 episodes",1,15,24 min.,24,24,Renewed,1,Children,English,2021-03-16T21:20:24.167-03:00,26,01,18,26-01-18
Motown Magic,childrens-animation,childrens-animation,20-Nov-18,"1 season, 25 episodes",1,25,15–26 min.,15,26,Renewed,1,Children,English,2021-03-16T21:20:24.167-03:00,20,11,18,20-11-18
3Below: Tales of Arcadia,childrens-animation,childrens-animation,21-Dec-18,"1 part, 13 episodes",0,13,23–24 min.,23,24,Renewed,1,Children,English,2021-03-16T21:20:24.167-03:00,21,12,18,21-12-18
Legend Quest,childrens-animation,childrens-animation,24-Feb-17,"1 season, 13 episodes",1,13,22 min.,22,22,Renewed,1,Children,English,2021-03-16T21:20:24.167-03:00,24,02,17,24-02-17
Pinky Malinky,childrens-animation,childrens-animation,1-Jan-19,"2 parts, 44 episodes",0,44,12–23 min.,12,23,Renewed,1,Children,English,2021-03-16T21:20:24.167-03:00,1,01,19,1-01-19


In [0]:
df2.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- GenreLabels: string (nullable = true)
 |-- Premiere: string (nullable = true)
 |-- Seasons: string (nullable = true)
 |-- SeasonsParsed: long (nullable = true)
 |-- EpisodesParsed: long (nullable = true)
 |-- Length: string (nullable = true)
 |-- MinLength: long (nullable = true)
 |-- MaxLength: long (nullable = true)
 |-- Status: string (nullable = true)
 |-- Active: long (nullable = true)
 |-- Table: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- dt_inclusao: string (nullable = true)
 |-- Premiere1: string (nullable = true)
 |-- Premiere2: string (nullable = true)
 |-- Premiere3: string (nullable = true)
 |-- FullPremiere: string (nullable = true)



In [0]:
# 3. Remover linhas duplicadas e trocar o resultado das linhas que tiverem a coluna "Seasons" de "TBA" para "a ser anunciado".

df3 = df2.dropDuplicates()
print("Distinct Seasons: " + str(df3.count()))
order = df3.sort(col("Active").desc(),col("Genre").desc())
display(order)

Distinct Seasons: 358


Title,Genre,GenreLabels,Premiere,Seasons,SeasonsParsed,EpisodesParsed,Length,MinLength,MaxLength,Status,Active,Table,Language,dt_inclusao,Premiere1,Premiere2,Premiere3,FullPremiere
Super Monsters Monster Party,chrildrens musical short,"chrildrens,musical,short",14-Sep-18,"1 season, 4 episodes",1,4,2 min.,2,2,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,14,09,18,14-09-18
True Tunes,chrildrens musical short,"chrildrens,musical,short",12-Jul-19,TBA,0,0,TBA,0,0,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,12,07,19,12-07-19
Dance & Sing with True,chrildrens musical short,"chrildrens,musical,short",18-May-18,"1 season, 11 episodes",1,11,2 min.,2,2,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,18,05,18,18-05-18
Spirit Riding Free,childrens-animation,childrens-animation,5-May-17,"8 seasons, 52 episodes",8,52,23 min.,23,23,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,5,05,17,5-05-17
Motown Magic,childrens-animation,childrens-animation,20-Nov-18,"1 season, 25 episodes",1,25,15–26 min.,15,26,Renewed,1,Children,English,2021-03-16T21:20:24.167-03:00,20,11,18,20-11-18
Spy Kids: Mission Critical,childrens-animation,childrens-animation,20-Apr-18,"2 seasons, 20 episodes",2,20,23–25 min.,23,23,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,20,04,18,20-04-18
Carmen Sandiego,childrens-animation,childrens-animation,18-Jan-19,"1 season, 9 episodes",1,9,23–33 min.,23,33,Renewed,1,Children,English,2021-03-16T21:20:24.167-03:00,18,01,19,18-01-19
Larva Island,childrens-animation,childrens-animation,19-Oct-18,"2 seasons, 26 episodes",2,26,8 min.,8,8,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,19,10,18,19-10-18
Pinky Malinky,childrens-animation,childrens-animation,1-Jan-19,"2 parts, 44 episodes",0,44,12–23 min.,12,23,Renewed,1,Children,English,2021-03-16T21:20:24.167-03:00,1,01,19,1-01-19
Stretch Armstrong and the Flex Fighters,childrens-animation,childrens-animation,17-Nov-17,"2 seasons, 23 episodes",2,23,23 min.,23,23,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,17,11,17,17-11-17


In [0]:
order.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- GenreLabels: string (nullable = true)
 |-- Premiere: string (nullable = true)
 |-- Seasons: string (nullable = true)
 |-- SeasonsParsed: long (nullable = true)
 |-- EpisodesParsed: long (nullable = true)
 |-- Length: string (nullable = true)
 |-- MinLength: long (nullable = true)
 |-- MaxLength: long (nullable = true)
 |-- Status: string (nullable = true)
 |-- Active: long (nullable = true)
 |-- Table: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- dt_inclusao: string (nullable = true)
 |-- Premiere1: string (nullable = true)
 |-- Premiere2: string (nullable = true)
 |-- Premiere3: string (nullable = true)
 |-- FullPremiere: string (nullable = true)



In [0]:
order1 = order.withColumn("Seasons", when(order.Seasons == "TBA","a ser anunciado").otherwise(order.Seasons))
display(order1)


Title,Genre,GenreLabels,Premiere,Seasons,SeasonsParsed,EpisodesParsed,Length,MinLength,MaxLength,Status,Active,Table,Language,dt_inclusao,Premiere1,Premiere2,Premiere3,FullPremiere
Super Monsters Monster Party,chrildrens musical short,"chrildrens,musical,short",14-Sep-18,"1 season, 4 episodes",1,4,2 min.,2,2,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,14,09,18,14-09-18
True Tunes,chrildrens musical short,"chrildrens,musical,short",12-Jul-19,a ser anunciado,0,0,TBA,0,0,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,12,07,19,12-07-19
Dance & Sing with True,chrildrens musical short,"chrildrens,musical,short",18-May-18,"1 season, 11 episodes",1,11,2 min.,2,2,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,18,05,18,18-05-18
Spirit Riding Free,childrens-animation,childrens-animation,5-May-17,"8 seasons, 52 episodes",8,52,23 min.,23,23,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,5,05,17,5-05-17
Motown Magic,childrens-animation,childrens-animation,20-Nov-18,"1 season, 25 episodes",1,25,15–26 min.,15,26,Renewed,1,Children,English,2021-03-16T21:20:24.167-03:00,20,11,18,20-11-18
Spy Kids: Mission Critical,childrens-animation,childrens-animation,20-Apr-18,"2 seasons, 20 episodes",2,20,23–25 min.,23,23,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,20,04,18,20-04-18
Carmen Sandiego,childrens-animation,childrens-animation,18-Jan-19,"1 season, 9 episodes",1,9,23–33 min.,23,33,Renewed,1,Children,English,2021-03-16T21:20:24.167-03:00,18,01,19,18-01-19
Larva Island,childrens-animation,childrens-animation,19-Oct-18,"2 seasons, 26 episodes",2,26,8 min.,8,8,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,19,10,18,19-10-18
Pinky Malinky,childrens-animation,childrens-animation,1-Jan-19,"2 parts, 44 episodes",0,44,12–23 min.,12,23,Renewed,1,Children,English,2021-03-16T21:20:24.167-03:00,1,01,19,1-01-19
Stretch Armstrong and the Flex Fighters,childrens-animation,childrens-animation,17-Nov-17,"2 seasons, 23 episodes",2,23,23 min.,23,23,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,17,11,17,17-11-17


In [0]:
# Filtro para identificar volume modificado.

filtro = order1.filter(order1.Seasons == "a ser anunciado")
display(filtro)

Title,Genre,GenreLabels,Premiere,Seasons,SeasonsParsed,EpisodesParsed,Length,MinLength,MaxLength,Status,Active,Table,Language,dt_inclusao,Premiere1,Premiere2,Premiere3,FullPremiere
True Tunes,chrildrens musical short,"chrildrens,musical,short",12-Jul-19,a ser anunciado,0,0,TBA,0,0,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,12,07,19,12-07-19
Exhibit A,True crime,"true,crime",28-Jun-19,a ser anunciado,0,0,TBA,0,0,Pending,1,DocuSeries,English,2021-03-16T21:20:24.167-03:00,28,06,19,28-06-19
The Chosen One,Thriller drama,"thriller,drama",28-Jun-19,a ser anunciado,0,0,TBA,0,0,Pending,1,ForeignLanguage,Portuguese,2021-03-16T21:20:24.167-03:00,28,06,19,28-06-19
Another Life,Science-fiction,science-fiction,25-Jul-19,a ser anunciado,0,0,TBA,0,0,Pending,1,Drama,English,2021-03-16T21:20:24.167-03:00,25,07,19,25-07-19
7 Seeds,Science fiction,science-fiction,28-Jun-19,a ser anunciado,0,0,TBA,0,0,Pending,1,Anime,English,2021-03-16T21:20:24.167-03:00,28,06,19,28-06-19
Love Alarm,Romantic comedy,"romantic,comedy",22-Aug-19,a ser anunciado,0,0,TBA,0,0,Pending,1,ForeignLanguage,Korean,2021-03-16T21:20:24.167-03:00,22,08,19,22-08-19
Typewriter,Horror,horror,19-Jul-19,a ser anunciado,0,0,TBA,0,0,Pending,1,ForeignLanguage,Hindi,2021-03-16T21:20:24.167-03:00,19,07,19,19-07-19
Sintonia,Drama,drama,9-Aug-19,a ser anunciado,0,0,TBA,0,0,Pending,1,ForeignLanguage,Portuguese,2021-03-16T21:20:24.167-03:00,9,08,19,9-08-19
The Naked Director,Drama,drama,8-Aug-19,a ser anunciado,0,0,TBA,0,0,Pending,1,ForeignLanguage,Japanese,2021-03-16T21:20:24.167-03:00,8,08,19,8-08-19
The Last Czars,Docu-series,docu-series,3-Jul-19,a ser anunciado,0,0,TBA,0,0,Pending,1,DocuSeries,English,2021-03-16T21:20:24.167-03:00,3,07,19,3-07-19


In [0]:
order1.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- GenreLabels: string (nullable = true)
 |-- Premiere: string (nullable = true)
 |-- Seasons: string (nullable = true)
 |-- SeasonsParsed: long (nullable = true)
 |-- EpisodesParsed: long (nullable = true)
 |-- Length: string (nullable = true)
 |-- MinLength: long (nullable = true)
 |-- MaxLength: long (nullable = true)
 |-- Status: string (nullable = true)
 |-- Active: long (nullable = true)
 |-- Table: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- dt_inclusao: string (nullable = true)
 |-- Premiere1: string (nullable = true)
 |-- Premiere2: string (nullable = true)
 |-- Premiere3: string (nullable = true)
 |-- FullPremiere: string (nullable = true)



In [0]:
# 4. Criar uma coluna nova chamada "Data de Alteração" e dentro dela um timestamp.

df4 = order1.withColumn("Data de Alteração", to_timestamp(current_timestamp(),"MM-dd-yyyy hh mm a"))\
      .withColumnRenamed("Data de Alteração","Data_de_Alteração")
#df4.show(1,truncate=False)
display(df4)

Title,Genre,GenreLabels,Premiere,Seasons,SeasonsParsed,EpisodesParsed,Length,MinLength,MaxLength,Status,Active,Table,Language,dt_inclusao,Premiere1,Premiere2,Premiere3,FullPremiere,Data_de_Alteração
Super Monsters Monster Party,chrildrens musical short,"chrildrens,musical,short",14-Sep-18,"1 season, 4 episodes",1,4,2 min.,2,2,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,14,09,18,14-09-18,2021-10-07T23:43:22.898+0000
True Tunes,chrildrens musical short,"chrildrens,musical,short",12-Jul-19,a ser anunciado,0,0,TBA,0,0,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,12,07,19,12-07-19,2021-10-07T23:43:22.898+0000
Dance & Sing with True,chrildrens musical short,"chrildrens,musical,short",18-May-18,"1 season, 11 episodes",1,11,2 min.,2,2,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,18,05,18,18-05-18,2021-10-07T23:43:22.898+0000
Spirit Riding Free,childrens-animation,childrens-animation,5-May-17,"8 seasons, 52 episodes",8,52,23 min.,23,23,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,5,05,17,5-05-17,2021-10-07T23:43:22.898+0000
Motown Magic,childrens-animation,childrens-animation,20-Nov-18,"1 season, 25 episodes",1,25,15–26 min.,15,26,Renewed,1,Children,English,2021-03-16T21:20:24.167-03:00,20,11,18,20-11-18,2021-10-07T23:43:22.898+0000
Spy Kids: Mission Critical,childrens-animation,childrens-animation,20-Apr-18,"2 seasons, 20 episodes",2,20,23–25 min.,23,23,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,20,04,18,20-04-18,2021-10-07T23:43:22.898+0000
Carmen Sandiego,childrens-animation,childrens-animation,18-Jan-19,"1 season, 9 episodes",1,9,23–33 min.,23,33,Renewed,1,Children,English,2021-03-16T21:20:24.167-03:00,18,01,19,18-01-19,2021-10-07T23:43:22.898+0000
Larva Island,childrens-animation,childrens-animation,19-Oct-18,"2 seasons, 26 episodes",2,26,8 min.,8,8,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,19,10,18,19-10-18,2021-10-07T23:43:22.898+0000
Pinky Malinky,childrens-animation,childrens-animation,1-Jan-19,"2 parts, 44 episodes",0,44,12–23 min.,12,23,Renewed,1,Children,English,2021-03-16T21:20:24.167-03:00,1,01,19,1-01-19,2021-10-07T23:43:22.898+0000
Stretch Armstrong and the Flex Fighters,childrens-animation,childrens-animation,17-Nov-17,"2 seasons, 23 episodes",2,23,23 min.,23,23,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,17,11,17,17-11-17,2021-10-07T23:43:22.898+0000


In [0]:
df4.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- GenreLabels: string (nullable = true)
 |-- Premiere: string (nullable = true)
 |-- Seasons: string (nullable = true)
 |-- SeasonsParsed: long (nullable = true)
 |-- EpisodesParsed: long (nullable = true)
 |-- Length: string (nullable = true)
 |-- MinLength: long (nullable = true)
 |-- MaxLength: long (nullable = true)
 |-- Status: string (nullable = true)
 |-- Active: long (nullable = true)
 |-- Table: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- dt_inclusao: string (nullable = true)
 |-- Premiere1: string (nullable = true)
 |-- Premiere2: string (nullable = true)
 |-- Premiere3: string (nullable = true)
 |-- FullPremiere: string (nullable = true)
 |-- Data_de_Alteração: timestamp (nullable = true)



In [0]:
# 5. Trocar os nomes das colunas de inglês para português, exemplo: "Title" para "Título" (com acentuação).

oldColumns = df4.schema.names
newColumns = ["Título","Gênero","GenreLabels","Pré estreia","Temporadas","SeasonsParsed","Episódios analisados"
              ,"Comprimento","Comprimento mínimo","Comprimento máximo","Status","Ativo","Mesa","Língua","dt_inclusao"
              ,"Premiere1","Premiere2","Premiere3","FullPremiere","Data_de_Alteração"]
df11 = reduce(lambda df4, colunas: df4.withColumnRenamed(oldColumns[colunas], newColumns[colunas]), range(len(oldColumns)), df4)

display(df11)



Título,Gênero,GenreLabels,Pré estreia,Temporadas,SeasonsParsed,Episódios analisados,Comprimento,Comprimento mínimo,Comprimento máximo,Status,Ativo,Mesa,Língua,dt_inclusao,Premiere1,Premiere2,Premiere3,FullPremiere,Data_de_Alteração
Super Monsters Monster Party,chrildrens musical short,"chrildrens,musical,short",14-Sep-18,"1 season, 4 episodes",1,4,2 min.,2,2,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,14,09,18,14-09-18,2021-10-07T23:46:28.016+0000
True Tunes,chrildrens musical short,"chrildrens,musical,short",12-Jul-19,a ser anunciado,0,0,TBA,0,0,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,12,07,19,12-07-19,2021-10-07T23:46:28.016+0000
Dance & Sing with True,chrildrens musical short,"chrildrens,musical,short",18-May-18,"1 season, 11 episodes",1,11,2 min.,2,2,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,18,05,18,18-05-18,2021-10-07T23:46:28.016+0000
Spirit Riding Free,childrens-animation,childrens-animation,5-May-17,"8 seasons, 52 episodes",8,52,23 min.,23,23,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,5,05,17,5-05-17,2021-10-07T23:46:28.016+0000
Motown Magic,childrens-animation,childrens-animation,20-Nov-18,"1 season, 25 episodes",1,25,15–26 min.,15,26,Renewed,1,Children,English,2021-03-16T21:20:24.167-03:00,20,11,18,20-11-18,2021-10-07T23:46:28.016+0000
Spy Kids: Mission Critical,childrens-animation,childrens-animation,20-Apr-18,"2 seasons, 20 episodes",2,20,23–25 min.,23,23,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,20,04,18,20-04-18,2021-10-07T23:46:28.016+0000
Carmen Sandiego,childrens-animation,childrens-animation,18-Jan-19,"1 season, 9 episodes",1,9,23–33 min.,23,33,Renewed,1,Children,English,2021-03-16T21:20:24.167-03:00,18,01,19,18-01-19,2021-10-07T23:46:28.016+0000
Larva Island,childrens-animation,childrens-animation,19-Oct-18,"2 seasons, 26 episodes",2,26,8 min.,8,8,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,19,10,18,19-10-18,2021-10-07T23:46:28.016+0000
Pinky Malinky,childrens-animation,childrens-animation,1-Jan-19,"2 parts, 44 episodes",0,44,12–23 min.,12,23,Renewed,1,Children,English,2021-03-16T21:20:24.167-03:00,1,01,19,1-01-19,2021-10-07T23:46:28.016+0000
Stretch Armstrong and the Flex Fighters,childrens-animation,childrens-animation,17-Nov-17,"2 seasons, 23 episodes",2,23,23 min.,23,23,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,17,11,17,17-11-17,2021-10-07T23:46:28.016+0000


In [0]:
df11.printSchema()

root
 |-- Título: string (nullable = true)
 |-- Gênero: string (nullable = true)
 |-- GenreLabels: string (nullable = true)
 |-- Pré estreia: string (nullable = true)
 |-- Temporadas: string (nullable = true)
 |-- SeasonsParsed: long (nullable = true)
 |-- Episódios analisados: long (nullable = true)
 |-- Comprimento: string (nullable = true)
 |-- Comprimento mínimo: long (nullable = true)
 |-- Comprimento máximo: long (nullable = true)
 |-- Status: string (nullable = true)
 |-- Ativo: long (nullable = true)
 |-- Mesa: string (nullable = true)
 |-- Língua: string (nullable = true)
 |-- dt_inclusao: string (nullable = true)
 |-- Premiere1: string (nullable = true)
 |-- Premiere2: string (nullable = true)
 |-- Premiere3: string (nullable = true)
 |-- FullPremiere: string (nullable = true)
 |-- Data_de_Alteração: timestamp (nullable = true)



In [0]:
df11.dtypes

Out[172]: [('Título', 'string'),
 ('Gênero', 'string'),
 ('GenreLabels', 'string'),
 ('Pré estreia', 'string'),
 ('Temporadas', 'string'),
 ('SeasonsParsed', 'bigint'),
 ('Episódios analisados', 'bigint'),
 ('Comprimento', 'string'),
 ('Comprimento mínimo', 'bigint'),
 ('Comprimento máximo', 'bigint'),
 ('Status', 'string'),
 ('Ativo', 'bigint'),
 ('Mesa', 'string'),
 ('Língua', 'string'),
 ('dt_inclusao', 'string'),
 ('Premiere1', 'string'),
 ('Premiere2', 'string'),
 ('Premiere3', 'string'),
 ('FullPremiere', 'string'),
 ('Data_de_Alteração', 'timestamp')]

In [0]:
# 6. Testar e verificar se existe algum erro de processamento do spark e identificar onde pode ter ocorrido o erro.



In [0]:
data = df11.registerTempTable("data")
output = spark.sql("SELECT * FROM data")
display(output)

Título,Gênero,GenreLabels,Pré estreia,Temporadas,SeasonsParsed,Episódios analisados,Comprimento,Comprimento mínimo,Comprimento máximo,Status,Ativo,Mesa,Língua,dt_inclusao,Premiere1,Premiere2,Premiere3,FullPremiere,Data_de_Alteração
Super Monsters Monster Party,chrildrens musical short,"chrildrens,musical,short",14-Sep-18,"1 season, 4 episodes",1,4,2 min.,2,2,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,14,09,18,14-09-18,2021-10-07T23:46:58.043+0000
True Tunes,chrildrens musical short,"chrildrens,musical,short",12-Jul-19,a ser anunciado,0,0,TBA,0,0,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,12,07,19,12-07-19,2021-10-07T23:46:58.043+0000
Dance & Sing with True,chrildrens musical short,"chrildrens,musical,short",18-May-18,"1 season, 11 episodes",1,11,2 min.,2,2,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,18,05,18,18-05-18,2021-10-07T23:46:58.043+0000
Spirit Riding Free,childrens-animation,childrens-animation,5-May-17,"8 seasons, 52 episodes",8,52,23 min.,23,23,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,5,05,17,5-05-17,2021-10-07T23:46:58.043+0000
Motown Magic,childrens-animation,childrens-animation,20-Nov-18,"1 season, 25 episodes",1,25,15–26 min.,15,26,Renewed,1,Children,English,2021-03-16T21:20:24.167-03:00,20,11,18,20-11-18,2021-10-07T23:46:58.043+0000
Spy Kids: Mission Critical,childrens-animation,childrens-animation,20-Apr-18,"2 seasons, 20 episodes",2,20,23–25 min.,23,23,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,20,04,18,20-04-18,2021-10-07T23:46:58.043+0000
Carmen Sandiego,childrens-animation,childrens-animation,18-Jan-19,"1 season, 9 episodes",1,9,23–33 min.,23,33,Renewed,1,Children,English,2021-03-16T21:20:24.167-03:00,18,01,19,18-01-19,2021-10-07T23:46:58.043+0000
Larva Island,childrens-animation,childrens-animation,19-Oct-18,"2 seasons, 26 episodes",2,26,8 min.,8,8,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,19,10,18,19-10-18,2021-10-07T23:46:58.043+0000
Pinky Malinky,childrens-animation,childrens-animation,1-Jan-19,"2 parts, 44 episodes",0,44,12–23 min.,12,23,Renewed,1,Children,English,2021-03-16T21:20:24.167-03:00,1,01,19,1-01-19,2021-10-07T23:46:58.043+0000
Stretch Armstrong and the Flex Fighters,childrens-animation,childrens-animation,17-Nov-17,"2 seasons, 23 episodes",2,23,23 min.,23,23,Pending,1,Children,English,2021-03-16T21:20:24.167-03:00,17,11,17,17-11-17,2021-10-07T23:46:58.043+0000


In [0]:
output.write.format("csv").mode("overwrite").option("sep","\t").save("/opt/intellij/spark_datagree/data/my-tsv-file.csv")

In [0]:
  # 7. Criar apenas 1 . csv com as seguintes colunas que foram nomeadas anteriormente "Title, Genre, Seasons, Premiere, Language, Active, Status, dt_inclusao, Data de Alteração" as
# colunas devem estar em português com header e separadas por ";".

output.write.format("csv").mode("overwrite").option("sep","\t").save("/opt/intellij/spark_datagree/data/my-tsv-file.csv")

output.write.format("csv").save("/FileStore/tables/mydata_3.csv").option("sep", ";").option("header", "true")


---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<command-1881034451122870> in <module>
      1 # 7. Criar apenas 1 . csv com as seguintes colunas que foram nomeadas anteriormente "Title, Genre, Seasons, Premiere, Language, Active, Status, dt_inclusao, Data de Alteração" as
      2 # colunas devem estar em português com header e separadas por ";".
----> 3 output.write.format("csv").save("/FileStore/tables/mydata_3.csv")\
      4 .option("sep", ";")\
      5 .option("header", "true")

AttributeError: 'NoneType' object has no attribute 'option'